# Inference

This notebook is for development/testing of the inference server (`inference.py`).

It is similar to the `training/training.ipynb` notebook,
but uses a live page instead of one from the dataset.

In [ ]:
import awe.utils
awe.utils.init_notebook()

In [ ]:
import awe.data.set.live
import awe.training.versioning
import awe.training.params
import awe.training.trainer
awe.utils.reload('awe', exclude=['awe.data.glove'])

In [ ]:
# Load trained version from a checkpoint.
version = awe.training.versioning.Version.get_latest()
checkpoint = version.get_checkpoints()[-1]
params = awe.training.params.Params.load_version(version)
print(f'{params=}')
trainer = awe.training.trainer.Trainer(params)
trainer.init_features()
trainer.restore_checkpoint(checkpoint)
trainer.restore_features()
trainer.create_model()
trainer.restore_model()

In [ ]:
# Mock of a live page (full page would need visuals extracted by the Node.js
# extractor app which would be impractical).
page = awe.data.set.live.Page(
    index=0,
    url='http://google.com',
    html_text='<html><body><b>b</b><i>i</i></body></html>',
    visuals_data={"/html":{"/body":{"/b":{"/text()":{"box":[0, 0, 1, 1]}},"/i":{"/text()":{"box":[1, 1, 2, 2]}}}}}
)

In [ ]:
# Prepare features for the page.
run = trainer.create_run([page], desc='live')

In [ ]:
# Predict results for the page.
preds = trainer.predict(run)
preds

In [ ]:
# Show predictions.
trainer.decode(preds)